# Importing the libraries

In [1]:
import pandas as pd
import numpy as np
#! pip install GetOldTweets3
import GetOldTweets3 as got3
import datetime
import time as t
import re

# User Input
Please input your query & number of tweets here:


In [7]:
 # ENTER THE QUERYWORDS as a string, no case sensitivity, can use boolean operations
name_of_coin = "Ethereum"
coin_symbol = "ETH"
#####################################################################################

nTweets = 100000
date_limit = datetime.date(2020, 1 ,1)


## Gathering the tweets

In [8]:
def gathering_data(name_of_coin, coin_symbol, nTweets, date_limit):
  
    querywords = "{0} Crypto OR {0} Cryptocurrency OR {0} Coin OR {0} Token OR {1} Crypto "\
                 "OR {1} Cryptocurrency OR {1} Coin OR {1} Token OR ${1}".format(name_of_coin, coin_symbol)
    
    batch_number = nTweets // 7000 + 1
    tweets_per_batch = int(nTweets / batch_number)
    date = []
    username = []
    text = []
    hashtags = []
    retweets = []
    favorites = []
    mentions = []
    to = []
    
    upper_date = datetime.date.today() + datetime.timedelta(days=1)
    print(f'Date of first scraped tweet: {datetime.date.today()}')
    print(f"Maximum number of batches: {batch_number}")
    print('Starting scraping tweets...\n')
    print('-'*40)

    for i in range(batch_number):
        print("Scraping batch", i + 1, "...")
        tic = t.time()
    
        #Using GOT to gather the tweets
        tweetCriteria = got3.manager.TweetCriteria().setQuerySearch(querywords).setMaxTweets(tweets_per_batch).\
                        setUntil(upper_date.strftime("%Y-%m-%d")).setLang("en").setSince(date_limit.strftime("%Y-%m-%d"))
        
        try:
            get_tweet = got3.manager.TweetManager.getTweets(tweetCriteria)
    
        except:
            print("sleeping for 10 minutes\n")
            t.sleep(10 * 60)
            continue 
            
        print(len(get_tweet), "of", tweets_per_batch, "tweets downloaded in ", round(t.time() - tic,2) , "s")

    
        #Updating the lists that will serve to create the DF
        for tweet in get_tweet:
            date.append(tweet.date)
            username.append(tweet.username)
            text.append(tweet.text)
            hashtags.append(tweet.hashtags)
            retweets.append(tweet.retweets)
            favorites.append(tweet.favorites)
            mentions.append(tweet.mentions)
            to.append(tweet.to)
            
        if len(get_tweet) == 0:
            break
        
        print(f'Updated date of last tweet: {date[-1]}')
        #Modifying the upper_date for later requests
        upper_date = date[-1].date()
        if upper_date <= date_limit:
            print("Limit date reached.")
            break
        
        if i != batch_number - 1:
            if len(get_tweet) >= 5000:  
                print("sleeping for 7 minutes...\n")
                t.sleep(7 * 60)
            else:
                print("sleeping for 2 minute...\n")
                t.sleep(2 * 60)
        
        
    
    tweets = pd.DataFrame({"date":date,"username":username,"text":text,\
                      "hashtags":hashtags,"mentions":mentions,"retweets":retweets,"favorites":favorites,"to":to})

    
    filename = '../Data/Unprocessed_data/' + re.sub(' ', '_', name_of_coin) + '_' + str(len(tweets)) + ".csv"
 
    return tweets, filename


In [9]:
tweets, filename = gathering_data(name_of_coin, coin_symbol, nTweets, date_limit)
tweets.to_csv(filename, index=False)

Date of first scraped tweet: 2020-08-24
Maximum number of batches: 15
Starting scraping tweets...

----------------------------------------
Scraping batch 1 ...
6666 of 6666 tweets downloaded in  304.75 s
Updated date of last tweet: 2020-08-09 19:42:05+00:00
sleeping for 7 minutes...

Scraping batch 2 ...
6666 of 6666 tweets downloaded in  307.94 s
Updated date of last tweet: 2020-07-26 22:45:29+00:00
sleeping for 7 minutes...

Scraping batch 3 ...
6666 of 6666 tweets downloaded in  318.68 s
Updated date of last tweet: 2020-07-10 13:10:03+00:00
sleeping for 7 minutes...

Scraping batch 4 ...
6666 of 6666 tweets downloaded in  332.15 s
Updated date of last tweet: 2020-06-22 20:37:52+00:00
sleeping for 7 minutes...

Scraping batch 5 ...
6666 of 6666 tweets downloaded in  335.8 s
Updated date of last tweet: 2020-06-01 16:46:10+00:00
sleeping for 7 minutes...

Scraping batch 6 ...
3683 of 6666 tweets downloaded in  177.82 s
Updated date of last tweet: 2020-05-19 23:40:34+00:00
sleeping for